In [ ]:
######################################## Import Library ########################################

%matplotlib qt
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob

################################################################################################
################################ First Step: Camera Calibration ################################
################################################################################################

# 1.0 use set of chessboard pictures to calibrate camera.

# prepare object points
nx = 9#TODO: enter the number of inside corners in x (column)
ny = 6#TODO: enter the number of inside corners in y (row)

# Make a list of calibration images
images = glob.glob('camera_cal/calibration*.jpg')
objpoints = []
imgpoints = []


objp = np.zeros((ny*nx,3),np.float32)
objp[:,:2] = np.mgrid[0:nx,0:ny].T.reshape(-1,2)
##print(objp)

for fname in images:
    img = cv2.imread(fname)

# Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

# Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (nx, ny), None)

# If found, draw corners
    if ret == True:
        imgpoints.append(corners)
        objpoints.append(objp)
        #Draw and display the corners
        #cv2.drawChessboardCorners(img, (nx, ny), corners, ret)
        #imgplot = plt.imshow(img)
        ##print(imgpoints)

ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)


################################################################################################
################################ Second Step: Finding Laneline #################################
################################################################################################

# Note: Please use mpimg.imread for image import

# 2.0 Region thresholding - crop image
def cropimg(img):
    mask = np.zeros_like(img[:,:,0])   
    ignore_mask_color = 255   
    imshape = img.shape

    vertices = np.array([[(imshape[1]*1/21,imshape[0]),(imshape[1]*7/20, imshape[0]*8/12),
                  (imshape[1]*13/20, imshape[0]*8/12), 
                  (imshape[1]*20/21,imshape[0])]], dtype=np.int32)
        
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    croppedimg = img.copy()
    croppedimg[mask!=255] = (0,0,0)
    return croppedimg

# 2.1 HLS color thresholding
def hls_select(img, threshh=(0, 255), threshl=(0, 255), threshs=(0, 255)):
    
    # 1) Convert img to HLS color space
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    H = hls[:,:,0]
    L = hls[:,:,1]
    S = hls[:,:,2]
    Lmax = max(L.flatten())
    Smax = max(S.flatten())
    
    # 2) Apply a threshold to the S channel
    binary = np.zeros_like(S)
    binary[(((H > threshh[0]) & (H <= threshh[1])) | ((L/Lmax > threshl[0]) & 
           (L/Lmax <= threshl[1]))) & (S/Smax > threshs[0]) & (S/Smax <= threshs[1])] = 1
    
    return binary

# 2.2 RGB color thresholding
def rgb_select(image):
   
    white_threshold_r = 200
    white_threshold_g = 200
    white_threshold_b = 200
    yellow_threshold_r = 180
    yellow_threshold_g = 140
    yellow_threshold_b = 120
    rgb_thresholds = np.zeros_like(image[:,:,0])

    whilteline_threshold = [white_threshold_r, white_threshold_g, white_threshold_b]
    yellowline_threshold = [yellow_threshold_r, yellow_threshold_g, yellow_threshold_b]

    # Mask pixels below the whilteline_threshold
    rgb_thresholds_w = (image[:,:,0] > whilteline_threshold[0]) & \
                        (image[:,:,1] > whilteline_threshold[1]) & \
                        (image[:,:,2] > whilteline_threshold[2])

    rgb_thresholds_y = (image[:,:,2] > yellowline_threshold[0]) & \
                        (image[:,:,1] > yellowline_threshold[1]) & \
                        (image[:,:,0] < yellowline_threshold[2])

    rgb_thresholds[(rgb_thresholds_w == True) | (rgb_thresholds_y == True)] = 1
    
    return rgb_thresholds
        
# 2.3 Combined function to find laneline for each image
def findlaneline(image):
    
    # 0) resize the image to 1280*720
    #resized_image = cv2.resize(image, (1280, 720)) 
    
    # 1) undistort the image
    undistimg = cv2.undistort(image, mtx, dist, None, mtx)

    # 2) crop the image to Left and Right side.
    croppedimg = cropimg(undistimg)

    # 3) find laneline pixels with HLS thresholding
    hls_binary = hls_select(croppedimg, threshh=(10, 35), threshl=(200/255, 1), threshs=(90/255, 1))
      
    # 4) find laneline pixels with RGB thresholding
    rgb_binary = rgb_select(croppedimg)
    #testa = np.dstack((rgb_binary, rgb_binary, rgb_binary)) * 255
    #testb = cv2.cvtColor(croppedimg_R, cv2.COLOR_RGB2HLS)
    #testtest = np.concatenate((croppedimg_R, testa), axis=1)
    #plt.imshow(testtest)
    
    # 5) combined laneline pixels
    combined = np.zeros_like(hls_binary)
    combined[(rgb_binary == 1)|(hls_binary == 1)] = 1
    return combined


################################################################################################
############################ Third Step: Transfer to bird eye view #############################
################################################################################################

# 3.0 warp / inverse warp image
def warp(img,a):
    img_size = (img.shape[1],img.shape[0])
    src = np.float32(
        [[553.40,480.197],
         [732.44,480.197],
         [261.396,680.094],
         [1043.88,680.094]])
    dst = np.float32(
        [[300,180],
         [1000,180],
         [300,680],
         [1000,680]])

    if a == 0:
        M = cv2.getPerspectiveTransform(src, dst)
        warped = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)
        return warped      
    if a == 1:
        Minv = cv2.getPerspectiveTransform(dst, src)
        warpinv = cv2.warpPerspective(img, Minv, img_size, flags=cv2.INTER_LINEAR)
        return warpinv



################################################################################################
############################ Fourth Step: Fit poly and find radius #############################
################################################################################################

# 4.0 Search for lane pixels, if there is no existing poly fit.
def find_lane_pixels(binary_warped):
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    minpixforline = 300
    #plt.plot(histogram)
    
    # Create an output image to draw on and visualize the result
    #out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    if max(histogram[:midpoint]) > 100:
        leftx_base = np.argmax(histogram[:midpoint])
    else: 
        leftx_base = 300
    if max(histogram[midpoint:]) > 100:
        rightx_base = np.argmax(histogram[midpoint:]) + midpoint
    else:
        rightx_base = 1000

    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 100

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        # Identify the nonzero pixels in x and y within the window #
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        # If you found > minpix pixels, recenter next window on their mean position
        if (len(good_left_inds) >= minpix) and (len(good_right_inds) >= minpix):
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))
        if (len(good_left_inds) >= minpix) and (len(good_right_inds) < minpix): 
            rightx_current = rightx_current + np.int(np.mean(nonzerox[good_left_inds])) - leftx_current
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if (len(good_left_inds) < minpix) and (len(good_right_inds) >= minpix): 
            leftx_current = leftx_current + np.int(np.mean(nonzerox[good_right_inds])) - rightx_current
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
        
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    # lane is not found is there is less than minpixforline pixels found
    if len(leftx) >= minpixforline:
        leftfound = True
    else:
        leftfound = False
        
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    # lane is not found is there is less than minpixforline pixels found
    if len(rightx) >= minpixforline:
        rightfound = True
    else:
        rightfound = False

    return leftx, lefty, leftfound, rightx, righty, rightfound

# 4.1 Search around existing laneline
def search_around_poly(binary_warped, left_fit, right_fit):
    # HYPERPARAMETER
    # Choose the width of the margin around the previous polynomial to search
    # The quiz grader expects 100 here, but feel free to tune on your own!
    margin = 100
    minpixforline = 300
    
    # Grab activated pixels
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    ### TO-DO: Set the area of search based on activated x-values ###
    ### within the +/- margin of our polynomial function ###
    ### Hint: consider the window areas for the similarly named variables ###
    ### in the previous quiz, but change the windows to our new search area ###
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + 
                    left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + 
                    left_fit[1]*nonzeroy + left_fit[2] + margin)))
    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + 
                    right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + 
                    right_fit[1]*nonzeroy + right_fit[2] + margin)))
    
    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    # lane is not found is there is less than minpixforline pixels found
    if len(leftx) >= minpixforline:
        leftfound = True
    else:
        leftfound = False
        
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    # lane is not found is there is less than minpixforline pixels found
    if len(rightx) >= minpixforline:
        rightfound = True
    else:
        rightfound = False
    
    return leftx, lefty, leftfound, rightx, righty, rightfound

# 4.2 Fit the laneline with poly fit.
def fit_polynomial(xset,yset,Line):
    Line.updatecurv = 0
    # Fit a second order polynomial to each using `np.polyfit`
    fittedpara = np.polyfit(yset, xset, 2)
    # If line failed, take whatever para we found
    if Line.side == 0:
        diff = abs(Rightline.fittedpara[0]-fittedpara[0])
    if Line.side == 1:
        diff = abs(Leftline.fittedpara[0]-fittedpara[0])   
    if Line.failed == True:
        Line.fittedpara = fittedpara
        Line.updatecurv = 1
    # Else, take para only when it did not have abrupt change from previous value and left/right lane curve parallel
    if abs(Line.fittedpara[0]-fittedpara[0])<=0.002 and diff<=0.0006 and Line.failed == False:
        Line.fittedpara = Line.fittedpara + 0.25* (fittedpara - Line.fittedpara)
        Line.updatecurv = 1
    return

# 4.3 Calculate curvature
def curvature(xset,yset,Line):
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    
    # Fit a second order polynomial to pixel positions in each fake lane line
    Fit_cr = np.polyfit(yset*ym_per_pix, xset*xm_per_pix, 2)
    
    curverad = ((1 + (2*Fit_cr[0]*image.shape[0]*ym_per_pix + Fit_cr[1])**2)**1.5) / np.absolute(2*Fit_cr[0])
    if curverad <= 1500:
        if Line.failed == True or Line.radius_of_curvature == 0:
            Line.radius_of_curvature = curverad
        if Line.updatecurv == 1 and Line.failed == False:
            Line.radius_of_curvature = Line.radius_of_curvature + 0.25* (curverad - Line.radius_of_curvature)
    return

# 4.4 Plot on image
def plotimg(image,mode):
    # Generate x and y values for plotting
    ploty = np.linspace(0, image.shape[0]-1, image.shape[0] )

    try:
        if mode == 0:
            left_fitx = Leftline.fittedpara[0]*ploty**2 + Leftline.fittedpara[1]*ploty + Leftline.fittedpara[2]
            right_fitx = Rightline.fittedpara[0]*ploty**2 + Rightline.fittedpara[1]*ploty + Rightline.fittedpara[2]
        if mode == 1:
            left_fitx = Leftline.fittedpara[0]*ploty**2 + Leftline.fittedpara[1]*ploty + Leftline.fittedpara[2]
            right_fitx = left_fitx + 700
        if mode == 2:
            right_fitx = Rightline.fittedpara[0]*ploty**2 + Rightline.fittedpara[1]*ploty + Rightline.fittedpara[2]
            left_fitx = right_fitx - 700

    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty

    ## Visualization ##
    # Add line center, warp is done from [300,680] to [1000,680], so find center at y=680.
    Leftline.center=(left_fitx[679]+right_fitx[679])/2
    # Plot between 2 fitted poly lines
    left_ready = left_fitx.astype(int)
    right_ready = right_fitx.astype(int)
    for i in range(image.shape[0]):
        for j in range(max(left_ready[i],0),min((right_ready[i]-1),1280)):
            
            image[i,j][1] = 180
            
    #image[ploty.astype(int), left_fitx.astype(int)] = [255,255,255]
    #image[ploty.astype(int), right_fitx.astype(int)] = [255,255,255]
    return image



################################################################################################
########################## Final Step: Test area / Actual processing ###########################
################################################################################################

# 5.0 Define a Line class to track the detected line
class Line():
    def __init__(self):
        # was the line detected in the last iteration?
        self.detected = False  
        # how many frames has the line been not detected?
        self.failcount = 0
        # was the line failed?
        self.failed = True     
        # last confirmed line parameter
        self.fittedpara = np.array([0,0,0])
        #radius of curvature of the line
        self.radius_of_curvature = 0.0
        #center of the line
        self.center = 0
        #side of the line
        self.side = 0
        # whether update curvature or not
        self.updatecurv = 0
        
# 5.1 Process Image
def processimage(image):
    laneline = findlaneline(image)
    binary_warped = warp(laneline,0)
    result = np.dstack((binary_warped, binary_warped, binary_warped))
    
    # If both side failed, find lane with sliding-window
    if Leftline.failed == True or Rightline.failed == True:
        leftx, lefty, leftfound, rightx, righty, rightfound = find_lane_pixels(binary_warped)
        # If both side found lane, try fit the lane with poly.
        if leftfound == True and rightfound == True:
            # hightlight left lane and fit it with poly
            result[lefty, leftx] = [255, 0, 0]
            fit_polynomial(leftx,lefty,Leftline)
            curvature(leftx,lefty,Leftline)
            Leftline.detected = True
            Leftline.failcount = 0
            Leftline.failed = False
            # hightlight right lane and fit it with poly
            result[righty, rightx] = [0, 0, 255]
            fit_polynomial(rightx,righty,Rightline)
            curvature(rightx,righty,Rightline)
            Rightline.detected = True
            Rightline.failcount = 0
            Rightline.failed = False
            # plot and return result image
            result = plotimg(result,0)
        if leftfound == True and rightfound == False:
            # hightlight left lane and fit it with poly
            result[lefty, leftx] = [255, 0, 0]
            result[righty, rightx] = [0, 0, 255]
            fit_polynomial(leftx,lefty,Leftline)
            curvature(leftx,lefty,Leftline)
            Leftline.detected = True
            Leftline.failcount = 0
            Leftline.failed = False
            # plot and return result image
            result = plotimg(result,1)
            # print('Right Lane failed')
        if leftfound == False and rightfound == True:
            # hightlight right lane and fit it with poly
            result[lefty, leftx] = [255, 0, 0]
            result[righty, rightx] = [0, 0, 255]
            fit_polynomial(rightx,righty,Rightline)
            curvature(rightx,righty,Rightline)
            Rightline.detected = True
            Rightline.failcount = 0
            Rightline.failed = False
            # plot and return result image
            result = plotimg(result,2)
            # print('Left Lane failed')
        if leftfound == False and rightfound == False:
            print('FAILUER')
                
    else:
        leftx, lefty, leftfound, rightx, righty, rightfound = search_around_poly(binary_warped,Leftline.fittedpara,Rightline.fittedpara)
        #print('search_around',leftfound,rightfound)
        # If both side found lane, try fit the lane with poly.
        if leftfound == True and rightfound == True:
            # hightlight left lane and fit it with poly
            result[lefty, leftx] = [255, 0, 0]
            fit_polynomial(leftx,lefty,Leftline)
            curvature(leftx,lefty,Leftline)
            Leftline.detected = True
            Leftline.failcount = 0
            Leftline.failed = False
            # hightlight right lane and fit it with poly
            result[righty, rightx] = [0, 0, 255]
            fit_polynomial(rightx,righty,Rightline)
            curvature(rightx,righty,Rightline)
            Rightline.detected = True
            Rightline.failcount = 0
            Rightline.failed = False
            # plot and return result image
            result = plotimg(result,0)
        if leftfound == True and rightfound == False:
            # hightlight left lane and fit it with poly
            result[lefty, leftx] = [255, 0, 0]
            result[righty, rightx] = [0, 0, 255]
            fit_polynomial(leftx,lefty,Leftline)
            curvature(leftx,lefty,Leftline)
            Leftline.detected = True
            Leftline.failcount = 0
            Leftline.failed = False
            # Add 1 to failcount to right lane
            Rightline.detected = False
            if Rightline.failcount == 20:
                Rightline.failcount = 0
                Rightline.failed = True
            else:
                Rightline.failcount = Rightline.failcount + 1
            # plot and return result image
            result = plotimg(result,0)
            # print('Right Lane missing')
        if leftfound == False and rightfound == True:
            # Add 1 to failcount to left lane
            Leftline.detected = False
            if Leftline.failcount == 20:
                Leftline.failcount = 0
                Leftline.failed = True
            else:
                Leftline.failcount = Leftline.failcount + 1
            # hightlight right lane and fit it with poly
            result[lefty, leftx] = [255, 0, 0]
            result[righty, rightx] = [0, 0, 255]
            fit_polynomial(rightx,righty,Rightline)
            curvature(rightx,righty,Rightline)
            Rightline.detected = True
            Rightline.failcount = 0
            Rightline.failed = False
            # plot and return result image
            result = plotimg(result,0)
            # print('Left Lane missing')
        if leftfound == False and rightfound == False:
            result[lefty, leftx] = [255, 0, 0]
            result[righty, rightx] = [0, 0, 255]
            # Add 1 to failcount to left lane
            Leftline.detected = False
            if Leftline.failcount == 20:
                Leftline.failcount = 0
                Leftline.failed = True
            else:
                Leftline.failcount = Leftline.failcount + 1
            # Add 1 to failcount to right lane
            Rightline.detected = False
            if Rightline.failcount == 20:
                Rightline.failcount = 0
                Rightline.failed = True
            else:
                Rightline.failcount = Rightline.failcount + 1
            result = plotimg(result,0)
            # print('Both Lane missing')
    
    # unwarp the final filter
    invwarpresult = warp(result,1)
    
    # visualize based on the filter
    add = np.zeros_like(image)
    add[invwarpresult[:,:,1]==180] = (0,255,0)
    copiedimgage = image.copy()
    copiedimgage[invwarpresult[:,:,0]==255] = (255,80,80)
    copiedimgage[invwarpresult[:,:,2]==255] = (0,153,255)
    
    # add original image and the visualized filter to 1 image
    final_result = cv2.addWeighted(copiedimgage, 1, add, 0.3, 0)
    
    # Calculate the curvature and display on the image
    a = Leftline.failed == True or Leftline.radius_of_curvature == 0
    b = Rightline.failed == True or Rightline.radius_of_curvature == 0
    if (a and b):
        radius = 'N/A'
    if not a and b:
        radius = str(int(Leftline.radius_of_curvature))+'m'
    if a and not b:
        radius = str(int(Rightline.radius_of_curvature))+'m'
    if not a and not b:
        radius = str(int((Leftline.radius_of_curvature + Rightline.radius_of_curvature)/2))+'m'

    # Calculate the center and display on the image
    if Leftline.center == 0:
        center = 'N/A'
    if Leftline.center != 0 and Leftline.center >= 700:
        center = '%.2f' % ((Leftline.center-700)*3.7/700) +'m left'
    if Leftline.center != 0 and Leftline.center < 700:
        center = '%.2f' % ((700-Leftline.center)*3.7/700) +'m right'
        
    # Diagnostic message
    diag = 'left found:'+str(len(leftx))+' '+str(leftfound)+' right found:'+str(len(rightx))+' '+str(rightfound)
    
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(final_result,'Radius of Curvature is: '+radius,(320,50),font,1,(255,255,255),2,cv2.LINE_AA)
    cv2.putText(final_result,'Vehicle is: '+center+' from center',(320,100),font,1,(255,255,255),2,cv2.LINE_AA)
    # Comment below to disable diag message
    #cv2.putText(final_result,diag,(320,150),font,1,(255,255,255),2,cv2.LINE_AA)
    #cv2.putText(final_result,str(Leftline.fittedpara[0])+' '+str(Rightline.fittedpara[0]),(100,200),font,1,(255,255,255),2,cv2.LINE_AA)
        
    laneline_1 = np.dstack((laneline, laneline, laneline)) * 255
    binary_warped_1 = np.dstack((binary_warped, binary_warped, binary_warped)) * 255
    
    result_1 = np.concatenate((image, laneline_1), axis=1)
    result_2 = np.concatenate((result, final_result), axis=1)
    result_3 = np.concatenate((result_1, result_2), axis=0)
    
    
    return result_3,final_result


################################################################################################
####################################### Main: Main code ########################################
################################################################################################

# Main Code:
cap = cv2.VideoCapture('project_video.mp4')
print('loading video')
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
out = cv2.VideoWriter('output_video/video_with_comparison/project_video.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 25, (frame_width*2,frame_height*2))
out2 = cv2.VideoWriter('output_video/project_video.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 25, (frame_width,frame_height))


Leftline = Line()
Rightline = Line()
Rightline.side = 1

while(True):
    ret, image = cap.read()
    if ret == True: 
        RGBimage = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        result,final_result = processimage(RGBimage)
        RGB_result = cv2.cvtColor(result, cv2.COLOR_RGB2BGR)
        RGB_final_result = cv2.cvtColor(final_result, cv2.COLOR_RGB2BGR)
        out.write(RGB_result)
        out2.write(RGB_final_result)
    
    else:
        print("finished.")
        break 
        
cap.release()
out.release()
out2.release()

    